In [ ]:
! pip install praw

     |████████████████████████████████| 167 kB 7.4 MB/s 
     |████████████████████████████████| 68 kB 6.1 MB/s 


### Creation of a dataset using tree structure

_The Praw library extracts Reddit posts in the following order: all posts from the creation of the thread, replies to the creator and then replies to the creator's replies, etc. The goal is to obtain a contextualized dataset of up to 6 posts prior to the extracted post._ 

In [ ]:
import praw
from praw.models import MoreComments
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd


reddit = praw.Reddit(
    client_id="Ij3g665KjrZLew",
    client_secret="aWEtDM8Y855CGkm34NqGWvBedpGdnQ",
    user_agent="android:com.Example.myredditapp:v1.2.3 (by u/Key_Bench_3896)",
)

subs = reddit.subreddit("france").hot(limit=200)
final=[]
results=[]
ids=[]
parent_ids=[]

for submission in subs :
    submission.comments.replace_more(limit=None)
    comment_queue = submission.comments[:]  # Seed with top-level
    while comment_queue:
        comment = comment_queue.pop(0)
        results.append([comment.body,comment.id,comment.parent_id[3:],comment.score,submission.title])
        comment_queue.extend(comment.replies)
    final.append(results)
    results=[]

print(final)

ids=[]
delet=[]

for k in range (len(final)):
    for i, e in reversed(list(enumerate(final[k]))):
        for j in range(len(final[k])-1):
            if final[k][i][2] == final[k][j][1]: #if parent_id of j is equal to comment id of i we update the final list
                final[k][i].append(final[k][j])
                delet.append(final[k][j])
    print(final)
    print(delet)
    for i in delet:
        if i in final[k]:
            indice=final[k].index(i)
            del final[k][indice]

print(final)

def depthCount(x):
    return 1 + max(map(depthCount, x)) if x and isinstance(x, list) else 0

number_of_replies_for_the_subs=[]
number_of_replies_for_a_sub=[]
for k in range(len(final)):
    for i in range(len(final[k])):
        number_of_replies_for_a_sub.append(depthCount(final[k][i]))
    number_of_replies_for_the_subs.append(number_of_replies_for_a_sub)
    number_of_replies_for_a_sub=[]

print(number_of_replies_for_the_subs)

topics_dict = {"title": [], "score": [], "comment": [], "context1": [], \
               "context2": [], "context3": [], "context4": [], "context5": []}

for k in range(len(final)):
    for i in range(len(final[k])):
        if number_of_replies_for_the_subs[k][i] == 1:
            topics_dict["title"].append(final[k][i][4])
            topics_dict["score"].append(final[k][i][3])
            topics_dict["comment"].append(final[k][i][0])
            for i in range(1,6):
                topics_dict["context" + str(i)].append(None)
        elif number_of_replies_for_the_subs[k][i] == 2:
            topics_dict["title"].append(final[k][i][4])
            topics_dict["score"].append(final[k][i][3])
            topics_dict["comment"].append(final[k][i][0])
            topics_dict["context1"].append(final[k][i][5][0])
            for i in range(2, 6):
                topics_dict["context" + str(i)].append(None)
        elif number_of_replies_for_the_subs[k][i] == 3:
            topics_dict["title"].append(final[k][i][4])
            topics_dict["score"].append(final[k][i][3])
            topics_dict["comment"].append(final[k][i][0])
            topics_dict["context1"].append(final[k][i][5][0])
            topics_dict["context2"].append(final[k][i][5][5][0])
            for i in range(3, 6):
                topics_dict["context" + str(i)].append(None)
        elif number_of_replies_for_the_subs[k][i] == 4:
            topics_dict["title"].append(final[k][i][4])
            topics_dict["score"].append(final[k][i][3])
            topics_dict["comment"].append(final[k][i][0])
            topics_dict["context1"].append(final[k][i][5][0])
            topics_dict["context2"].append(final[k][i][5][5][0])
            topics_dict["context3"].append(final[k][i][5][5][5][0])
            for i in range(4, 6):
                topics_dict["context" + str(i)].append(None)
        elif number_of_replies_for_the_subs[k][i] == 5:
            topics_dict["title"].append(final[k][i][4])
            topics_dict["score"].append(final[k][i][3])
            topics_dict["comment"].append(final[k][i][0])
            topics_dict["context1"].append(final[k][i][5][0])
            topics_dict["context2"].append(final[k][i][5][5][0])
            topics_dict["context3"].append(final[k][i][5][5][5][0])
            topics_dict["context4"].append(final[k][i][5][5][5][5][0])
            topics_dict["context5"].append(None)
        elif number_of_replies_for_the_subs[k][i] == 6:
            topics_dict["title"].append(final[k][i][4])
            topics_dict["score"].append(final[k][i][3])
            topics_dict["comment"].append(final[k][i][0])
            topics_dict["context1"].append(final[k][i][5][0])
            topics_dict["context2"].append(final[k][i][5][5][0])
            topics_dict["context3"].append(final[k][i][5][5][5][0])
            topics_dict["context4"].append(final[k][i][5][5][5][5][0])
            topics_dict["context5"].append(final[k][i][5][5][5][5][5][0])
        else:
            topics_dict["title"].append(None)
            topics_dict["score"].append(None)
            topics_dict["comment"].append(None)
            topics_dict["context1"].append(None)
            topics_dict["context2"].append(None)
            topics_dict["context3"].append(None)
            topics_dict["context4"].append(None)
            topics_dict["context5"].append(None)

topics_data = pd.DataFrame(topics_dict)
topics_data.head(10)
topics_data.to_csv('TreeStructure.csv', index=False)

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readt

### Reddit Dataset without Tree Structure

_Unfortunately most of the posts extracted do not get a context of a minimum length of 5 which leads to a too small dataset to be exploited properly. Here is a more classical extraction of the conversations from the /france subreddit. The context is however lost._



In [ ]:
import praw
from praw.models import MoreComments
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd


reddit = praw.Reddit(
    client_id="Ij3g665KjrZLew",
    client_secret="aWEtDM8Y855CGkm34NqGWvBedpGdnQ",
    user_agent="android:com.Example.myredditapp:v1.2.3 (by u/Key_Bench_3896)",
)

subs = reddit.subreddit("/france").hot(limit=5000)
final=[]
results=[]
ids=[]
parent_ids=[]

for submission in subs :
    submission.comments.replace_more(limit=None)
    comment_queue = submission.comments[:]  # Seed with top-level
    while comment_queue:
        comment = comment_queue.pop(0)
        results.append([comment.body,comment.id,comment.parent_id[3:],comment.score,submission.title])
        comment_queue.extend(comment.replies)
    final.append(results)
    results=[]


topics_dict = {"title": [], "score": [], "comment": []}

for k in range(len(final)):
    for i in range(len(final[k])):
      topics_dict["title"].append(final[k][i][4])
      topics_dict["score"].append(final[k][i][3])
      topics_dict["comment"].append(final[k][i][0])

topics_data = pd.DataFrame(topics_dict)
topics_data.head(10)
topics_data.to_csv('Test1.csv', index=False)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous envi

In [ ]:
topics_data = pd.DataFrame(topics_dict)
topics_data.head(10000)

,title,score,comment
0,Mercredi Tech - 2021-07-21,3,[montre running]\nC'est moi qui ouvre le bal a...
1,Mercredi Tech - 2021-07-21,3,[Répéteur wifi]\n\nBonjour ! Dans la maison de...
2,Mercredi Tech - 2021-07-21,3,y'a de l'echo sur ce sujet ...
3,Mercredi Tech - 2021-07-21,2,"Hello ! Pour débuter le Mercredi Tech, je vais..."
4,Mercredi Tech - 2021-07-21,2,"[DOCK POUR DISQUE DUR]\n\nHello tous, je reten..."
...,...,...,...
9995,Forum Libre - 2021-07-20,1,Novak Djokovic est aussi végétalien
9996,Forum Libre - 2021-07-20,2,"Tu me rassures, je suis à peu près au même niv..."
9997,Forum Libre - 2021-07-20,1,Dune ça fait 50ans qu'ils nous le tease (en tr...
9998,Forum Libre - 2021-07-20,6,> je porte ma tante\n\npeut-on dire que *tu so...


## Dialogue and Extract and Reddit Dataset formatting

_Putting in the format of a dataframe the database of conversations aimed at learning French as well as the data extracted from Reddit._

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


fichier = open("/content/DialogueAndExtractDataset.txt", "r")

dialogs=[]
dialog_line=[]

for ligne in fichier:
    if len(ligne) > 1:
        if '--' in ligne:
            dialogs.append(dialog_line)
            dialog_line = []
        elif ':' in ligne[:15]:
            indice=ligne.index(':')
            line=ligne[indice+1:].rstrip('\n')
            dialog_line.append(line)
        else:
            x=0

print(dialogs)

topics_dict = {"response": [], "context": [], "context0": [], "context1": [], \
               "context2": [], "context3": [], "context4": [], "context5": []}
contexted = []
n=7

for k in range(len(dialogs)):
    for i in range(n,len(dialogs[k])):
        row=[]
        prev = i - 1 - n
        for j in range(i, prev, -1):
            row.append(dialogs[k][j])
        contexted.append(row)

print(len(contexted))


columns = ['response', 'context']
columns = columns + ['context/'+str(i) for i in range(n-1)]

df1 = pd.DataFrame.from_records(contexted, columns=columns)

fichier = open("/content/Reddit.txt", "r")
fichier2 = open("/content/EnglishCommonWords.txt", "r")

stopWords = ['imbecile', 'putain',' imbecile', ' imbécile', ' imbecile', ' imbécile', ' abruti', ' abruti', ' abruti', ' andouille', ' andouille', ' andouille', ' andouille', ' andouille', ' andouille', ' andouille', ' andouille', ' andouille', ' andouille', ' andouille', ' andouille', ' avorton', ' avorton', ' imbecile', ' imbecile', ' imbécile', ' imbecile', ' imbécile', ' espingoin', ' espingoin', ' espingoin', ' espingoin', ' espingoin', ' espingoin', ' emmanche', ' emmanche', ' emmanché', ' emmerder', ' emmerdeuse', ' emmerdeur', ' emmerdeuse', ' emmanche', ' emmanche', ' emmanché', ' empafe', ' empafe', ' empafé', ' empapaoute', ' empapaoute', ' empapaouté', ' empapaoute', ' empapaoute', ' empapaouté', ' empafe', ' empafe', ' empafé', ' empapaoute', ' empapaoute', ' empapaouté', ' empapaoute', ' empapaoute', ' empapaouté', ' encule', ' enculer', ' enculeur', ' encule', ' enculé', ' enfoire', ' enfoire', ' enfoiré', ' enfoire', ' enfoire', ' enfoiré', ' enfoire', ' enfoire', ' enfoiré', ' enflure', ' enfoire', ' enfoire', ' enfoiré', ' enfoire', ' enfoire', ' enfoiré', ' enfoire', ' enfoire', ' enfoiré', ' envaselineur', ' envaselineur', ' envaselineur', ' envaselineur', ' envaselineur', ' envaselineur', ' envaselineur', ' envaselineur', ' envaselineur', ' envaselineur', ' envaselineur', ' envaselineur', ' espingoin', ' espingoin', ' espingoin', ' espingoin', ' espingoin', ' espingoin', ' etron', ' etron', ' sagouin', ' sagouin', ' sagouin', ' sagouin', ' sagouin', ' sagouin', ' salaud', ' salop', ' salopard', ' salope', ' salope', ' sale', ' sale', ' salop', ' salopard', ' salope', ' salope', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' sottiseux', ' sottiseux', ' sottiseux', ' sottiseux', ' sottiseux', ' sottiseux', ' sous-merde', ' sous-merde', ' sent-la-pisse', ' sent-la-pisse', ' sent-la-pisse', ' sent-la-pisse', ' sent-la-pisse', ' sent-la-pisse', ' sagouin', ' sagouin', ' sagouin', ' sagouin', ' sagouin', ' sagouin', ' salop', ' salope', ' salopard', ' salope', ' sale', ' salaud', ' sale', ' salop', ' salope', ' salopard', ' salope', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' schbeb', ' schbeb', ' schleu', ' schleu', ' schnoc', ' schnock', ' schnoque', ' schnoque', ' schnoc', ' schnock', ' schnoque', ' schnoque', ' sent-la-pisse', ' sent-la-pisse', ' sent-la-pisse', ' sent-la-pisse', ' sent-la-pisse', ' sent-la-pisse', ' sottiseux', ' sottiseux', ' sottiseux', ' sottiseux', ' sottiseux', ' sottiseux', ' sous-merde', ' sous-merde', ' stearique', ' stearique', ' stearique', ' stearique', ' stearique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' abruti', ' abruti', ' abruti', ' aller se faire foutre', ' allez', ' va t’en', ' amène ta mère pourque je te refasse', ' andouille', ' andouille', ' andouille', ' andouille', ' andouille', ' andouille', ' andouille', ' andouille', ' andouille', ' andouille', ' andouille', ' andouille', ' avorton', ' avale mes couilles', ' avorton', ' biatch', ' biatch', ' bicot', ' bicot', ' bite', ' bitembois', ' bitembois', ' bitembois', ' bitembois', ' bite', ' bitembois', ' bitembois', ' bitembois', ' bitembois', ' batard', ' bordel', ' bordel', ' bouffon', ' bougnoul', ' Bougnoulie', ' bougnoulisation', ' bougnouliser', ' bougnoulisation', ' bougnouliser', ' Bougnoulie', ' bougnoulisation', ' bougnouliser', ' bougnoulisation', ' bougnouliser', ' Bougnoulie', ' bougnoulisation', ' bougnouliser', ' bougnoulisation', ' bougnouliser', ' Bougnoulie', ' bougnoulisation', ' bougnouliser', ' bougnoulisation', ' bougnouliser', ' bougnoule', ' bougnoule', ' Bougnoulie', ' bougnoulisation', ' bougnouliser', ' bougnoulisation', ' bougnouliser', ' Bougnoulie', ' bougnoulisation', ' bougnouliser', ' bougnoulisation', ' bougnouliser', ' bougre', ' bougre', ' boukak', ' boukak', ' bounioul', ' bounioul', ' bounioul', ' bourdille', ' bourdille', ' bourdille', ' bourdille', ' bourdille', ' bourdille', ' bouseux', ' bouseux', ' biatch', ' biatch', ' bicot', ' bicot', ' bite', ' bitembois', ' bitembois', ' bitembois', ' bitembois', ' bite', ' bitembois', ' bitembois', ' bitembois', ' bitembois', ' beauf', ' beauf', ' bachie coupe', ' baise', ' baise moi mor', ' baise toi', ' baiser', ' bander', ' bâtard', ' batard', ' bâtard', ' beauf', ' beauf', ' biatch', ' biatch', ' bicot', ' bicot', ' bigornette', ' bise', ' bite', ' bitembois', ' bitembois', ' bitembois', ' bitembois', ' bite', ' bitembois', ' bitembois', ' bitembois', ' bitembois', ' bitte', ' bloblos', ' bon sang', ' bordel', ' bordel', ' bordel de merde', ' bosser', ' bouffon', ' bougnoul', ' Bougnoulie', ' bougnoulisation', ' bougnouliser', ' bougnoulisation', ' bougnouliser', ' Bougnoulie', ' bougnoulisation', ' bougnouliser', ' bougnoulisation', ' bougnouliser', ' Bougnoulie', ' bougnoulisation', ' bougnouliser', ' bougnoulisation', ' bougnouliser', ' Bougnoulie', ' bougnoulisation', ' bougnouliser', ' bougnoulisation', ' bougnouliser', ' bougnoule', ' bougnoule', ' Bougnoulie', ' bougnoulisation', ' bougnouliser', ' bougnoulisation', ' bougnouliser', ' Bougnoulie', ' bougnoulisation', ' bougnouliser', ' bougnoulisation', ' bougnouliser', ' bougre', ' bougre', ' boukak', ' boukak', ' bounioul', ' bounioul', ' bounioul', ' bourdille', ' bourdille', ' bourdille', ' bourdille', ' bourdille', ' bourdille', ' bourré', ' bourrée', ' bouseux', ' bouseux', ' brise-burnes', ' brise-burnes', ' brise-burnes', ' brise-burnes', ' branler', ' branleur', ' branler', ' branleur', ' branque', ' branque', ' brise-burnes', ' brise-burnes', ' brise-burnes', ' brise-burnes', ' brackmard', ' branler', ' branleur', ' branlage', ' branler', ' branlette', ' branleur', ' branleur de chiens morts', ' branleuse', ' branque', ' branque', ' brise-burnes', ' brise-burnes', ' brise-burnes', ' brise-burnes', ' brouter le cresson', ' broutte-minou', ' brûle en enfer', " brule en l'enfer", ' brûle en l’enfer', " c'est des conneries", ' c’est chiant', ' c’est des conneries', ' c’est le bordel ici', ' c’est quoi ce bordel', ' c’est vraiment de ta faute', ' casse-bonbon', ' casse-bonbon', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-bonbon', ' casse-bonbon', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' cacou', ' cacou', ' cafre', ' cafre', ' caldoche', ' caldoche', ' caldoche', ' caldoche', ' casse-bonbon', ' casse-bonbon', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-bonbon', ' casse-bonbon', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' coche', ' coche', ' con', ' conasse', ' conasse', ' conard', ' conasse', ' conasse', ' cons', ' conasse', ' conasse', ' conard', ' conasse', ' conasse', ' conchier', ' conchier', ' conchier', ' conchier', ' conchier', ' conchier', ' connasse', ' connasse', ' connard', ' connarde', ' connarde', ' connasse', ' connasse', ' conne', ' connasse', ' connasse', ' connard', ' connarde', ' connarde', ' connasse', ' connasse', ' conne', ' cons', ' couillon', ' couillonner', ' couille', ' couilles', ' couillon', ' couillonner', ' couillonner', ' couille', ' couilles', ' couille', ' couilles', ' counifle', ' counifle', ' counifle', ' counifle', ' counifle', ' counifle', ' courtaud', ' courtaud', ' ça casse les couilles', ' ça craint', ' ça fait chier', ' ça fait shier', " ça m'emmerde", ' ça me casse les couilles', ' ça me fait chier', ' ça me saoule', ' ça pue', ' ça schlingue', ' casse-bonbon', ' casse-bonbon', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-bonbon', ' casse-bonbon', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' cacou', ' caca', ' cacou', ' cafre', ' cafre', ' cailler', ' caldoche', ' caldoche', ' caldoche', ' caldoche', ' calice', ' calisse', ' casse-bonbon', ' casse-bonbon', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-bonbon', ' casse-bonbon', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-couille', ' casse-couilles', ' casse-toi', ' casse-toi', ' idiot', ' casse-toi', ' idiote', ' casser les couilles', ' cave', ' ce putain d’embouteillage', ' ce putain d’ordinateur', ' cela me soûle', ' chiennasse', ' chiennasse', ' chiennasse', ' chiennasse', ' chier', ' chiennasse', ' chiennasse', ' chiennasse', 'Starfoullah',' chiennasse', ' chier', ' chinetoc', 'Putain',' chinetoque', ' chinetoc', ' chinetoque', ' chinetoc', ' chinetoque', ' chinetoc', ' chinetoque', ' chintok', ' chintok', ' chachar', ' chagasse', ' chagasse', ' chagasse', ' chagasse', ' chauffard', ' chiennasse', ' chiennasse', ' chiennasse', ' chiennasse', ' chier', ' chieur', ' chieurs', ' chiennasse', ' chiennasse', ' chiennasse', ' chiennasse', ' chier', ' chieur', ' chieurs', ' chinetoc', ' chinetoque', ' chinetoc', ' chinetoque', ' chinetoc', ' chinetoque', ' chinetoc', ' chinetoque', ' chintok', ' chintok', ' chachar', ' chagasse', ' chagasse', ' chagasse', ' chagasse', ' charogne', ' chatte', ' chauffard', ' chiennasse', ' chiennasse', ' chiennasse', ' chiennasse', ' chier', ' chieur', ' chieurs', ' chiant', ' chiante', ' chiasse', ' chien sale', ' chiennasse', ' chiennasse', ' chiennasse', ' chiennasse', ' chienne', ' chier', ' chieur', ' chieurs', ' chieuse', ' chinetoc', ' chinetoque', ' chinetoc', ' chinetoque', ' chinetoc', ' chinetoque', ' chinetoc', ' chinetoque', ' chintok', ' chintok', ' chiottes', ' chleuh', ' chleuh', ' chnoque', ' chnoque', ' chnoque', ' chnoque', ' clito', ' clitoris', ' coche', ' coche', ' con', ' conasse', ' conasse', ' conard', ' conasse', ' conasse', ' cons', ' conasse', ' conasse', ' conard', ' conasse', ' conasse', ' conchier', ' conchier', ' conchier', ' conchier', ' conchier', ' conchier', ' connasse', ' connasse', ' connard', ' connarde', ' connarde', ' connasse', ' connasse', ' conne', ' connasse', ' connasse', ' connard', ' connarde', ' connarde', ' connasse', ' connasse', ' conne', ' connerie', ' conneries', ' cons', ' couille', ' couilles', ' couillon', ' couillonner', ' couille', ' couilles', ' couille', ' couilles', ' couillon', ' couillonner', ' couillonne', ' couillonner', ' counifle', ' counifle', ' counifle', ' counifle', ' counifle', ' counifle', ' courtaud', ' courtaud', ' cricri', ' crotte', ' crotte', ' crotté', ' crouillat', ' crouille', ' crouillat', ' crouille', ' crouillat', ' crouille', ' crouillat', ' crouille', ' crouillat', ' crouille', ' crouillat', ' crouille', ' croûton', ' cricri', ' cretin', ' cretin', ' cretin', ' crevard', ' crevard', ' crevure', ' cramouille', ' cretin', ' crétin', ' cretin', ' crétin', ' cretin', ' crétin', ' crevard', ' crevard', ' crève la gueule ouverte', ' crevure', ' cricri', ' criss', ' crisse', ' crosseur', ' crotte', ' crotte', ' crotté', ' crouillat', ' crouille', ' crouillat', ' crouille', ' crouillat', ' crouille', ' crouillat', ' crouille', ' crouillat', ' crouille', ' crouillat', ' crouille', ' croûton', ' cul', ' debile', ' debile', ' debile', ' deguelasse', ' deguelasse', ' demerder', ' debile', ' débile', ' debile', ' débile', ' debile', ' débile', ' debile', ' débile', ' debile', ' débile', ' debile', ' débile', ' déconne', ' déconner', ' degage', ' dégage', ' dégénéré', ' déguelasse', ' deguelasse', ' déguelasse', ' dégueulasse', ' demerder', ' démerder', ' dinde', ' drouille', ' drouille', ' drouille', ' drouille', ' drouille', ' drouille', ' drague', ' drouille', ' drouille', ' drouille', ' drouille', ' drouille', ' drouille', ' du schnoc', ' du schnoc', ' duschnock', ' duschnock', ' ducon', ' duconnot', ' ducon', ' duconnot', ' dugenoux', ' dugenoux', ' dugenoux', ' dugenoux', ' dugland', ' dugland', ' duschnock', ' duschnock', ' espingoin', ' espingoin', ' espingoin', ' espingoin', ' espingoin', ' espingoin', ' emmanche', ' emmanché', ' emmanche', ' emmanché', ' emmerdant', ' emmerder', ' emmerdeuse', ' emmerdeur', ' emmerdeuse', ' empafe', ' empafé', ' empapaoute', ' empapaouté', ' empapaoute', ' empapaouté', ' empafe', ' empafé', ' empapaoute', ' empapaouté', ' empapaoute', ' empapaouté', ' encule', ' enculé', ' encule de ta mère', ' enculé de ta mère', ' enculée', ' enculer', ' enculer les mouches', ' enculer une mouche', ' enculeur', ' enculeurs', ' enfoire', ' enfoiré', ' enfoire', ' enfoiré', ' enfoire', ' enfoiré', ' enflure', ' enfoire', ' enfoiré', ' enfoire', ' enfoiré', ' enfoire', ' enfoiré', ' enfoirée', ' envaselineur', ' envaselineur', ' envaselineur', ' envaselineur', ' envaselineur', ' envaselineur', ' envaselineur', ' envaselineur', ' envaselineur', ' envaselineur', ' envaselineur', ' envaselineur', ' epais', ' épais', ' epais', ' épais', ' epais', ' épais', ' epais', ' épais', ' epais', ' épais', ' epais', ' épais', ' epais', ' épais', ' epais', ' épais', ' epais', ' épais', ' epais', ' épais', ' epais', ' épais', ' epais', ' épais', ' espingoin', ' espingoin', ' espingoin', ' espingoin', ' espingoin', ' espingoin', ' etron', ' étron', ' etron', ' étron', ' fiotte', ' fiotte', ' fiotte', ' fiotte', ' fouteur', ' fouteur', ' foutre', ' foutre', ' fiotte', ' fiotte', ' fiotte', ' fiotte', ' feignasse', ' feignasse', ' feignasse', ' face de cul', ' faire enculer', ' fais chier', ' FDP', ' feignasse', ' feignasse', ' feignasse', ' ferme ta gueule', ' fiotte', ' fiotte', ' fiche moi le paix', ' fif', ' fille de pute', ' film de cul', ' fils de pute', ' fils de salope', ' fiotte', ' fiotte', ' folle', ' fous le camps', ' fous le camps et morte', ' fouteur', ' fouteur', ' foutre', ' foutre', ' foutre la merde', ' foutre le bordel', ' fritz', ' fritz', ' framble', ' fritz', ' fucker', ' fumier', ' fumier', ' fumier', ' fumier', ' fumier', ' fumier', ' garce', ' garce', ' gaupe', ' gaupe', ' Godon', ' gogol', ' goï', ' gouilland', ' gouilland', ' gouine', ' gouine', ' gouilland', ' gouilland', ' gouine', ' gouine', ' gouilland', ' gouilland', ' gouine', ' gouine', ' gourde', ' gourde', ' gourgandine', ' gourgandine', ' gourgandine', ' gourgandine', ' gourgandine', ' gourgandine', ' gourgandine', ' gourgandine', ' gourgandine', ' gourgandine', ' gourgandine', ' gourgandine', ' garage à bite', ' garce', ' garce', ' gaupe', ' gaupe', ' GDM', ' gerbe', ' gerber', ' gland', ' glandouillou', ' glandouillou', ' glandouillou', ' glandeuse', ' glandeur', ' glandeuse', ' glandeuse', ' glandeur', ' glandeuse', ' glandouillou', ' glandouillou', ' glandouillou', ' glandu', ' gland', ' glandouillou', ' glandouillou', ' glandouillou', ' glandeuse', ' glandeur', ' glandeuse', ' glandeuse', ' glandeur', ' glandeuse', ' glandouillou', ' glandouillou', ' glandouillou', ' glandu', ' gnoul', ' gnoule', ' gnoule', ' gnoul', ' gnoule', ' gnoule', ' Godon', ' gogol', ' goï', ' gouilland', ' gouilland', ' gouine', ' gouine', ' gouilland', ' gouilland', ' gouine', ' gouine', ' gouilland', ' gouilland', ' gouine', ' gouine', ' gourde', ' gourde', ' gourgandine', ' gourgandine', ' gourgandine', ' gourgandine', ' gourgandine', ' gourgandine', ' gourgandine', ' gourgandine', ' gourgandine', ' gourgandine', ' gourgandine', ' gourgandine', ' grognasse', ' grognasse', ' grognasse', ' grognasse', ' grognasse', ' grognasse', ' grognasse', ' grognasse', ' graine', ' grande folle', ' grognasse', ' grognasse', ' grognasse', ' grognasse', ' grognasse', ' grognasse', ' grognasse', ' grognasse', ' grogniasse', ' gros tas', ' gross cul', ' guindoule', ' guindoule', ' guindoule', ' guindoule', ' guindoule', ' guindoule', ' guindoule', ' guindoule', ' gueniche', ' gueniche', ' gueniche', ' gueniche', ' gueniche', ' gueniche', ' guenon', ' gueule', ' guindoule', ' guindoule', ' guindoule', ' guindoule', ' idiot', ' idiote', ' il est stupide', ' il fait chaud ici', ' enculé', ' il me fait chier', ' imbecile', ' imbecile', ' imbécile', ' imbecile', ' imbécile', " j'en ai marre", " j'en ai plus rien à foutre", " j'en ai ral le bol", " j'en ai ral le cul", " j'en ai rien à cirer", " j'en ai rien à foutre", ' j’ai baisé ta mère', ' j’ai envie de chier', ' j’en ai marre', ' j’en ai plus rien à foutre', ' j’en ai ral le bol', ' j’en ai ral le cul', ' j’en ai ras le bol', ' j’en ai rien à cirer', ' jean-foutre', ' jean-foutre', ' jean-foutre', ' jean-foutre', " je m'en fiche", " je m'en fou", " je m'en fous", " je m'en sacre", ' je m’emmerde', ' Je m’en bats les couilles', ' je m’en fiche', ' je m’en fou', ' je m’en fous', ' je me fais chier ici', ' je me fais chier.', ' je pis dans ta bouche', " je t'emmerde", " je t'encule", ' je t’emmerde', ' je te déteste', ' je te nique ta race sale fils de chien', ' je veux faire l’amour avec toi', ' jean-foutre', ' jean-foutre', ' jean-foutre', ' jean-foutre', ' jouir', ' kikoo', ' kikou', ' kikoo', ' kikou', ' kikoo', ' kikou', ' kikoo', ' kikou', ' kikoo', ' kikou', ' kikoo', ' kikou', ' Kraut', ' Kraut', ' lacheux', ' lacheux', ' lavette', ' lavette', ' lopette', ' lopette', ' la meme chose a toi', ' la putain de ta mère', ' la vache', ' lacheux', ' lâcheux', ' lacheux', ' lâcheux', ' laisse-moi tranquille', ' lavette', ' lavette', ' lèche-cul', ' lopette', ' lopette', ' miserable', ' miserable', ' misérable', ' miserable', ' misérable', ' miserable', ' misérable', ' miserable', ' misérable', ' michto', ' michto', ' minable', ' minable', ' minable', ' minable', ' minus', ' minus', ' miserable', ' miserable', ' miserable', ' misérable', ' miserable', ' misérable', ' miserable', ' misérable', ' miserable', ' misérable', ' magot', ' magot', ' makoume', ' makoume', ' makoumé', ' makoume', ' makoume', ' makoumé', ' manche', ' manche', ' mange-merde', ' mange-merde', ' marchandot', ' marchandot', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' mauviette', ' mauviette', ' mauviette', ' mauviette', ' mauviette', ' mauviette', ' moinaille', ' moinaille', ' moins-que-rien', ' moins-que-rien', ' moinaille', ' moinaille', ' moins-que-rien', ' moins-que-rien', ' moinaille', ' moinaille', ' moins-que-rien', ' moins-que-rien', ' moinaille', ' moinaille', ' moins-que-rien', ' moins-que-rien', ' moinaille', ' moinaille', 'merde','Merde',' moins-que-rien', ' moins-que-rien', ' moinaille', ' moinaille', ' moins-que-rien', ' moins-que-rien', ' monacaille', ' monacaille', ' monacaille', ' monacaille', ' monacaille', ' monacaille', ' monacaille', ' monacaille', ' monacaille', ' monacaille', ' monacaille', ' monacaille', ' moricaud', ' moricaud', ' moricaud', ' moricaud', ' moricaud', ' moricaud', ' miserable', ' miserable', ' miserable', ' misérable', ' miserable', ' misérable', ' miserable', ' misérable', ' miserable', ' misérable', ' michto', ' michto', ' minable', ' minable', ' minable', ' minable', ' minus', ' minus', ' merdaillon', ' merdaille', ' merdaillon', ' merdaillon', ' merdaille', ' merdaillon', ' merdaillon', ' merdaille', ' merdaillon', ' merdouillard', ' merdouillard', ' merdouillard', ' merdouillard', ' merdouillard', ' merdouillard', ' merde', ' merdeux', ' merdaillon', ' merdaille', ' merdaillon', ' merdaillon', ' merdaille', ' merdaillon', ' merdaillon', ' merdaille', ' merdaillon', ' merdouillard', ' merdouillard', ' merdouillard', ' merdouillard', ' merdouillard', ' merdouillard', ' ma couille', ' magot', ' magot', ' makoume', ' makoume', ' makoumé', ' makoume', ' makoume', ' makoumé', ' mal baisée', ' MALPT', ' mance', ' manche', ' manche', ' mange-merde', ' mange de la merde', ' mange ma bite', ' mange merde et morte', ' mange tes grands morts', ' mange-merde', ' marchandot', ' marchandot', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' margouilliste', ' maudit', ' mauviette', ' mauviette', ' mauviette', ' mauviette', ' mauviette', ' mauviette', ' ménage à trois', ' merdaillon', ' merdaille', ' merdaillon', ' merdaillon', ' merdaille', ' merdaillon', ' merdaillon', ' merdaille', ' merdaillon', ' merdouillard', ' merdouillard', ' merdouillard', ' merdouillard', ' merdouillard', ' merdouillard', ' merdaillon', ' merdaille', ' merdaillon', ' merdaillon', ' merdaille', ' merdaillon', ' merdaillon', ' merdaille', ' merdaillon', ' merdasse', ' merde', ' merde embulante', ' merdeuse', ' merdeux', ' merdouillard', ' merdouillard', ' merdouillard', ' merdouillard', ' merdouillard', ' merdouillard', ' mes couilles en toi', ' metteux', ' meuf', ' miserable', ' miserable', ' miserable', ' misérable', ' miserable', ' misérable', ' miserable', ' misérable', ' miserable', ' misérable', ' michto', ' michto', ' minable', ' minable', ' minable', ' minable', ' mince', ' minus', ' minus', ' miserable', ' miserable', ' miserable', ' misérable', ' miserable', ' misérable', ' miserable', ' misérable', ' miserable', ' misérable', ' mMeurs', ' pute', ' moinaille', ' moinaille', ' moins-que-rien', ' moins-que-rien', ' moinaille', ' moinaille', ' moins-que-rien', ' moins-que-rien', ' moinaille', ' moinaille', ' moins-que-rien', ' moins-que-rien', ' moinaille', ' moinaille', ' moins-que-rien', ' moins-que-rien', ' moinaille', ' moinaille', ' moins-que-rien', ' moins-que-rien', ' moinaille', ' moinaille', ' moins-que-rien', ' moins-que-rien', ' mon dieu', ' monacaille', ' monacaille', ' monacaille', ' monacaille', ' monacaille', ' monacaille', ' monacaille', ' monacaille', ' monacaille', ' monacaille', ' monacaille', ' monacaille', ' moricaud', ' moricaud', ' moricaud', ' moricaud', ' morceau de merde', ' moricaud', ' moricaud', ' niaiseux', ' niaiseux', ' niaiseux', ' niaiseux', ' niac', ' niakoue', ' niakoue', ' niakoué', ' niakoue', ' niakoue', ' niakoué', ' niaiseux', ' niaiseux', ' niaiseux', ' niaiseux', ' niac', ' niakoue', ' niakoue', ' niakoué', ' niakoue', ' niakoue', ' niakoué', ' nique', ' niquer', ' nique', ' niquer', ' nase', ' nase', ' naze', ' naze', ' niaiseux', ' niaiseux', ' niaiseux', ' niaiseux', ' niac', ' niakoue', ' niakoue', ' niakoué', ' niakoue', ' niakoue', ' niakoué', ' niaiseux', ' niaiseux', ' niaiseux', ' niaiseux', ' niac', ' niakoue', ' niakoue', ' niakoué', ' niakoue', ' niakoue', ' niakoué', ' nique', ' niquer', ' nique', ' niquer', ' negro', ' negro', ' nana', ' nase', ' nase', ' naze', ' naze', ' ne rien foutre', ' negro', ' négro', ' nègre', ' negro', ' négro', ' niac', ' niaiseux', ' niaiseux', ' niaiseux', ' niaiseux', ' niakoue', ' niakoue', ' niakoué', ' niakoue', ' niakoue', ' niakoué', ' niac', ' niaiseux', ' niaiseux', ' niaiseux', ' niaiseux', ' niakoue', ' niakoue', ' niakoué', ' niakoue', ' niakoue', ' niakoué', ' nique', ' niquer', ' nique', ' nique la police', ' nique ta mere', ' nique ta mère', ' nique ta mère espèce de gros bâtard de merde', ' nique ta mère la chauve', ' nique ta race', ' niquer', ' nom de dieu', ' nom de dieu de merde', ' nom de Dieu/mon Dieu', ' noune', ' NTM', ' oh la vache', ' oh putain de merde', ' oh', ' merde', ' osti', ' osti de calisse de tabarnak', ' ostie', ' pissou', ' pissou', ' pignouf', ' pignouf', ' pissou', ' pissou', ' pakos', ' pakos', ' pakos', ' pakos', ' panoufle', ' panoufle', ' panoufle', ' panoufle', ' patarin', ' patarin', ' patarin', ' porcas', ' porcasse', ' porcasse', ' porcas', ' porcasse', ' porcasse', ' porcas', ' porcasse', ' porcasse', ' porcas', ' porcasse', ' porcasse', ' poucav', ' poucav', ' pouf', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' pounde', ' pounde', ' poundé', ' pourriture', ' pourriture', ' pourriture', ' pourriture', ' pourriture', ' pourriture', ' pissou', ' pissou', ' pignouf', ' pignouf', ' pimbêche', ' pimbêche', ' pissou', ' pisseux', ' pisseux', ' pissou', ' pecque', ' pedale', ' pedoque', ' pede', ' pedale', ' pedoque', ' pequenaud', ' pequenaud', ' pet', ' petasse', ' petasse', ' peteux', ' petasse', ' petasse', ' pakos', ' pakos', ' pakos', ' pakos', ' palucher', ' panoufle', ' panoufle', ' panoufle', ' panoufle', ' partir en couille', ' patarin', ' patarin', ' patarin', ' PD', ' pecque', ' pedale', ' pédale', ' pedale', ' pédale', ' pedoque', ' pédoque', ' pedoque', ' pédoque', ' pedale', ' pédale', ' pedale', ' pédale', ' pede', ' pédé', ' pedoque', ' pédoque', ' pedoque', ' pédoque', ' pequenaud', ' péquenaud', ' pequenaud', ' péquenaud', ' pequenaud', ' péquenaud', ' pequenaud', ' péquenaud', ' pet', ' pétasse', ' petasse', ' pétasse', ' petasse', ' pétasse', ' petasse', ' pétasse', ' petasse', ' pétasse', ' petasse', ' pétasse', ' petasse', ' pétasse', ' petasse', ' pétasse', ' péter', ' péter la cenne', ' péter les couilles', ' peteux', ' péteux', ' petit tabernac', ' petite pute', ' pissou', ' pissou', ' pignouf', ' pignouf', ' pimbêche', ' pimbêche', ' pipi', ' pissou', ' pisseux', ' pisser', ' pisseux', ' pissou', ' plouc', ' pleutre', ' pleutre', ' plote', ' plotte', ' plouc', ' porcas', ' porcasse', ' porcasse', ' porcas', ' porcasse', ' porcasse', ' porcas', ' porcasse', ' porcasse', ' porcas', ' porcasse', ' porcasse', ' poucav', ' poucav', ' pouf', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffe', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' pouffiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' poufiasse', ' pounde', ' pounde', ' poundé', ' poupée', ' pourriture', ' pourriture', ' pourriture', ' pourriture', ' pourriture', ' pourriture', ' pousse-crotte', ' punaise', ' punaise', ' punaise', ' punaise', ' punaise', ' punaise', ' punaise', ' punaise', ' punaise', ' punaise', ' punaise', ' punaise', ' punaise', ' punaise', ' punaise', ' punaise', ' punaise', ' punaise', ' punaise', ' punaise', ' punaise', ' punaise', ' punaise', ' punaise', ' purée', ' putin', ' putain', ' putain', ' putain', ' putin', ' pute', ' putain', ' putain', ' putain', ' putain de bordel de merde', ' putain de merde', ' putain de temps', ' putain t’es moche', ' pute', ' putin', ' qu’est-ce que tu branles', ' queutard', ' queutard', ' quel bordel', ' quel salaud', ' queue', ' queutard', ' queutard', ' ripopee', ' ripopee', ' ripopée', ' ripopee', ' ripopée', ' ripopee', ' ripopee', ' ripopée', ' ripopee', ' ripopée', ' raclure', ' raclure', ' raton', ' raton', ' rosbif', ' rosbif', ' rosbif', ' robespierrot', ' robespierrot', ' robespierrot', ' robespierrot', ' robespierrot', ' robespierrot', ' robespierrot', ' robespierrot', ' robespierrot', ' robespierrot', ' robespierrot', ' robespierrot', ' rosbif', ' rosbif', ' rosbif', ' roulure', ' roulure', ' ripopee', ' ripopee', ' ripopée', ' ripopee', ' ripopée', ' ripopee', ' ripopee', ' ripopée', ' ripopee', ' ripopée', ' raclure', ' raclure', ' ramoner', ' raton', ' raton', ' rien à branler', ' rien à foutre', ' ripopee', ' ripopee', ' ripopée', ' ripopee', ' ripopée', ' ripopee', ' ripopee', ' ripopée', ' ripopee', ' ripopée', ' rosbif', ' rosbif', ' rosbif', ' robespierrot', ' robespierrot', ' robespierrot', ' robespierrot', ' robespierrot', ' robespierrot', ' robespierrot', ' robespierrot', ' robespierrot', ' robespierrot', ' robespierrot', ' robespierrot', ' rosbif', ' rosbif', ' rosbif', ' roulure', ' roulure', ' s’en branler', ' s’en foutre', ' sagouin', ' sagouin', ' sagouin', ' sagouin', ' sagouin', ' sagouin', ' salaud', ' salop', ' salopard', ' salope', ' saloperie', ' saloperie', ' salope', ' sale', ' sale', ' salop', ' salopard', ' salope', ' saloperie', ' salope', ' saloperie', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' sottiseux', ' sottiseux', ' sottiseux', ' sottiseux', ' sottiseux', ' sottiseux', ' sous-merde', ' sous-merde', ' sent-la-pisse', ' sent-la-pisse', ' sent-la-pisse', ' sent-la-pisse', ' sent-la-pisse', ' sent-la-pisse', ' sa mère', ' sa mère la pute', ' sa race', ' sac à foutre', ' sac à merde', ' sac merde', ' sac vin', ' sacrament', ' sacrebleu', ' sagouin', ' sagouin', ' sagouin', ' sagouin', ' sagouin', ' sagouin', ' salop', ' salope', ' salopard', ' salope', ' saloperie', ' sale', ' salaud', ' sale', ' salop', ' salope', ' saloperie', ' salopard', ' salope', ' saloperie', ' saloperie', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' satrouille', ' schbeb', ' schbeb', ' schleu', ' schleu', ' schnoc', ' schnock', ' schnoque', ' schnoque', ' schnoc', ' schnock', ' schnoque', ' schnoque', ' se foutre dans la merde', ' se taire', ' sent-la-pisse', ' sent-la-pisse', ' sent-la-pisse', ' sent-la-pisse', ' sent-la-pisse', ' sent-la-pisse', ' sottiseux', ' sottiseux', ' sottiseux', ' sottiseux', ' sottiseux', ' sottiseux', ' sous-merde', ' sous-merde', ' stearique', ' stearique', ' stearique', ' stearique', ' stearique', ' stearique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' stearique', ' stéarique', ' stupide', ' suce', ' suceuse', " t'as pas de couilles", " t'es rien qu'un petit connard", " t'es un salaud", ' t‘es rien qu’un petit connard', ' t’as intérêt ne pas m’énervé', ' t’as pas de couilles', ' t’es débile', ' t’es rien qu’un petit connard', ' t’es un salaud', ' t’es vraiment emmerdant', ' toi', ' tafiole', ' tafiole', ' tafiole', ' tafiole', ' tafiole', ' tafiole', ' tafiole', ' tafiole', ' tafiole', ' tafiole', ' tafiole', ' tafiole', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouze', ' tantouze', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouze', ' tantouze', ' tapette', ' tapette', ' tarlouze', ' tarlouze', ' tarlouze', ' tarlouze', ' tocard', ' tocard', ' tebe', ' tebe', ' tebé', ' teteux', ' teube', ' teube', ' teubé', ' ta gueule', ' ta mère elle suce des ours', ' ta mere est francaise', ' ta mère est une pute', ' ta mere la pute', ' ta mère la pute', ' ta mere suce des bites en enfer', ' tabarnak', ' tafiole', ' tafiole', ' tafiole', ' tafiole', ' tafiole', ' tafiole', ' tafiole', ' tafiole', ' tafiole', ' tafiole', ' tafiole', ' tafiole', ' tais-toi', ' tanche', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouze', ' tantouze', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouserie', ' tantouze', ' tantouze', ' tapette', ' tapette', ' tarlouze', ' tarlouze', ' tarlouze', ' tarlouze', ' tarte', ' tas de marde', ' tas de merde', ' tata', ' tebe', ' tebé', ' teteux', ' téteux', ' tête de nœud', ' teteux', ' téteux', ' teube', ' teubé', ' teuch', ' teuf', ' tocard', ' tocard', ' trainee', ' trainee', ' trainee', ' trainee', ' trainee', ' traînee', ' traînee', ' traînée', ' trainee', ' traînee', ' traînée', ' trouduc', ' trainee', ' trainee', ' trainee', ' trainee', ' trainee', ' traînee', ' traînee', ' traînée', ' trainee', ' traînee', ' traînée', ' très stupide', ' tringler', ' trique', ' troncher', ' trou de cul', ' trou du cul', ' trouduc', ' truiasse', ' truiasse', ' truiasse', ' truiasse', ' truiasse', ' truiasse', ' truiasse', ' truiasse', ' truiasse', ' truiasse', ' truiasse', ' truiasse', ' truiasse', ' truiasse', ' truiasse', ' truiasse', ' truiasse', ' truiasse', ' truiasse', ' truiasse', ' truiasse', ' truiasse', ' truiasse', ' truiasse', ' tu commence m’enmerder', ' tu es con', ' tu es très stupide', ' tu es un putain', ' tu et un ane', ' tu fais chier', " tu m'emmerdes", ' tu m’emmerdes', ' tu me fatigues', ' tu me gonfles', ' turlute', ' une salope', ' viedase', ' viedase', ' viedase', ' viedase', ' vier', ' vide-couilles', ' vide-couilles', ' vide-couilles', ' vide-couilles', ' vide-couilles', ' vide-couilles', ' vide-couilles', ' vide-couilles', ' viédase', ' viedase', ' viédase', ' viedase', ' viédase', ' viedase', ' viédase', ' viedase', ' viédase', ' viedase', ' viédase', ' viedase', ' viédase', ' viedase', ' viédase', ' vier', ' vaurien', ' vaurien', ' vaurien', ' vaurien', ' vaurien', ' vaurien', ' viedase', ' viedase', ' viedase', ' viedase', ' vier', ' vide-couilles', ' vide-couilles', ' vide-couilles', ' vide-couilles', ' vide-couilles', ' vide-couilles', ' vide-couilles', ' vide-couilles', ' viedase', ' viédase', ' viedase', ' viédase', ' viedase', ' viédase', ' viedase', ' viédase', ' viedase', ' viédase', ' viedase', ' viédase', ' viedase', ' viédase', ' viedase', ' viédase', ' vier', ' va te crosser', ' va te faire', ' va te faire baiser', ' va te faire enculer', ' va te faire foutre', ' va te faire mettre', ' va te faire voir', ' vache', ' vas faire foutre a la vache', ' vaurien', ' vaurien', ' vaurien', ' vaurien', ' vaurien', ' vaurien', ' veuve', ' viedase', ' viedase', ' viedase', ' viedase', ' vier', ' vide-couilles', ' vide-couilles', ' vide-couilles', ' vide-couilles', ' vide-couilles', ' vide-couilles', ' vide-couilles', ' vide-couilles', ' viedase', ' viédase', ' viedase', ' viédase', ' viedase', ' viédase', ' viedase', ' viédase', ' viedase', ' viédase', ' viedase', ' viédase', ' viedase', ' viédase', ' viedase', ' viédase', ' vier', ' vous êtes stupide', ' vous sentez comme le boeuf et le fromage', ' vous vissez', ' xeropineur', ' xeropineur', ' xeropineur', ' xeropineur', ' xeropineur', ' xeropineur', ' xeropineur', ' xéropineur', ' xeropineur', ' xéropineur', ' xeropineur', ' xéropineur', ' xeropineur', ' xéropineur', ' xeropineur', ' xéropineur', ' xeropineur', ' xéropineur', ' xeropineur', ' xéropineur', ' xeropineur', ' xéropineur', ' xeropineur', ' xéropineur', ' xeropineur', ' xéropineur', ' xeropineur', ' xéropineur', ' xeropineur', ' xéropineur', ' youd', ' youpin', ' youpinisation', ' youpinisation', ' youpinisation', ' youpinisation', ' youpine', ' youpine', ' youpin', ' youpinisation', ' youpinisation', ' youpinisation', ' youpinisation', ' youpine', ' youpine', ' youpin', ' youpine', ' youpine', ' youpinisation', ' youpinisation', ' youpinisation', ' youpinisation', ' youtre', ' youtre', ' yeule', ' yeule', ' youd', ' youpin', ' youpinisation', ' youpinisation', ' youpinisation', ' youpinisation', ' youpine', ' youpine', ' youpin', ' youpinisation', ' youpinisation', ' youpinisation', ' youpinisation', ' youpine', ' youpine', ' youpin', ' youpine', ' youpine', ' youpinisation', ' youpinisation', ' youpinisation', ' youpinisation', ' youtre', ' youtre', ' zguegue', ' zguegue', ' zguègue', ' zguegue', ' zguègue', ' zigounette', ' zizi', 'pd', 'khey']


englishwords=[]
contenu=fichier2.readlines()
for ligne in contenu:
    ligne=ligne.replace('\n'," ")
    ligne=ligne.replace("!!","")
    if len(ligne)>3:
      englishwords.append(ligne)

dialogs=[]
content=fichier.readlines()

for ligne in content:
  english=False
  stop=False
  ligne=ligne.replace('\n'," ")
  for word in englishwords:
    if word in ligne:
      english=True
  for word in stopWords:
    if word in ligne:
      stop=True
  if english==False and stop==False and ligne != '[removed]':
    dialogs.append(ligne)

print(dialogs)

topics_dict = {"response": [], "context": [], "context0": [], "context1": [], \
               "context2": [], "context3": [], "context4": [], "context5": []}
contexted = []
n=7

for i in range(len(dialogs)):
        row=[]
        prev = i - 1 - n
        for j in range(i, prev, -1):
            row.append(dialogs[j])
        contexted.append(row)

print(len(contexted))


columns = ['response', 'context']
columns = columns + ['context/'+str(i) for i in range(n-1)]

df2 = pd.DataFrame.from_records(contexted, columns=columns)
df=df1.append(df2)
df.head(1000)

trn_df, val_df = train_test_split(df, test_size = 0.2)
trn_df.head(10000)

[[' Vous avez des questions ?', ' Oui, je ne comprends pas très bien le sens de« vers huit heures».', ' « Vers huit heures » ? Ça signifie « autour de huit  heures» . Ce n’est pas à huit heures exactement. Tu comprends ?', ' Oui, merci.', ' Une autre question ?', ' Oui, j’ai une question à propos du texte numéro 2. Ligne 10. C’est écrit « j’aurais voulu un thé ». Mais, pourquoi c’est « J’aurais voulu » ?', ' Alors, quand on commande quelque chose, on utilise souvent le verbe vouloir. On  a le choix entre plusieurs formes : Je veux – je voudrais – j’aurais voulu.', ' Est-ce que « j’aurais voulu » est le plus poli ?', ' Oui et non. Je pense qu’on utilise vraiment les deux sans faire de distinction. On veut juste marquer la politesse. Tu comprends.', ' Oui, c’est très clair maintenant.', ' Il y a d’autres questions ? … Non ? Alors, je vous donne les devoirs pour demain. Vous ferez les exercices 4 et 5 p. 27.', ' 4  ..', ' Oui, 4 et 5 page 27. Merci, à demain.', ' À demain !'], [' Qu’est-c

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
539,Qui a dis qu’il y aurait un après covid,"Ça passera, aucun doute là dessus.",Singacontre,Bonne analyse..,Enfin!,"Ça, c'est uniquement si la loi passe le Parlem...",Ca s'applique aussi si c'est un resto pseudo-p...,Pas trop fan de cette serie de vidéos excessiv...
1736,"Le pluriel, en français, commence à deux.""Pas ...",Flemme de répondre à toutes ces âneries.,Merci. Je pensais être le seul à penser ça.,On appelle ça le bonapartisme.,"C'est ça. ""quoi qu'il en coute"".",Foutagedegueuletisme 😛,"Qu'enestilisme ça claque, j'aime bien","C'est vrai qu'on est comme des frères, mais pa..."
388,C’est un endroit magnifique ! Surtout depuis ...,"J’imagine. Et c’était comment, alors ?","Si, c’était noir de monde, surtout que c’étai...","En août ? Mais, il n’y avait pas trop de monde ?",En août !,Vous y êtes allés quand ?,"Oui, j’y suis allée l’année dernière avec Jac...",Tu es déjà allée au Mont-Saint-Michel ?
5481,Tu es expat dans quel pays?,"""pays en phase d'islamisation"" Yikes","Jure tant que tu veux, je ne te crois pas.",Et je vous jure que ce n’est pas un cas isolé.,Je te crois,"Les français interdit sur métro, le comble.",Je trouve ça plutôt bien d'être le centre d'in...,Ouais depuis notre défaite à l'Euro je prends ...
2232,C'est simple. Les tests antigéniques sont acce...,T'inquiète pas.,Excellente carrément,Sinkabawtitte!,Vitemadose répertorie les créneaux dispo de pl...,Tu prends rendez-vous sur Doctolib.fr ou maiia...,"Ou alors ton public est vraiment bête, vraimen...",Tous les pays occidentaux ont plus ou moins le...
...,...,...,...,...,...,...,...,...
4874,J’ai pas compris en fait,J'ai ri,Elle est bonne celle là!,Je valide avec une validation validée,"Dans le respect des gestes barrières, la dista...","J'AVIONS VNE QVESTION, ESTE QVE CECI ESTOY RET...",JCVD?,"""Il semple""Hé, c'est mon papi qui a traduit !"
3454,C’était une blague en fait …,👀,r/jechiedansmonfroc,"Vous êtes tous des menteurs, je suis le vrai K...",Tu m'as fumé,Non,"Han... T'es pas drôle gros, t'aurais pu répond...",Kaamelott*T'tention y'en a qui sont tatillons !
4699,C'est en cuisine comme dans beaucoup d'autres ...,"Par des Alsaciens,le goût est très proche",ENORMISSIME,r/transfertsdegrandmere,Je ne comprends pas la blague. On ne voit pas ...,Haha,"Un très jeune Prince, certains sont livré déjà...",ṔÁ ÁUvidpa aummpa aunt
2040,j'ai eu 20 erreur 404 donc non ca marchait pas...,"Enfin, les gens comprennent. On va enfin se dé...","Bravo, La France (vous me manque. Sigh)",Il n'y a malheureusement que la menace qui fon...,> c'ESt pas comme Ça qu'On Va LES CONVaINcREBe...,"u/Verethra, on en parlait hier, je crois qu'il...",[deleted],Macron a eu le nez creux. Ne pas pouvoir sorti...


### Dataset Books and Interviews

_Extraction for the Interviews and Books datasets._

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


fichier = open("/content/Entretiens.txt", "r")

dialogs=[]
dialog_line=[]

for ligne in fichier:
    if len(ligne) > 1:
        if '--' in ligne:
            dialogs.append(dialog_line)
            dialog_line = []
        elif ':' in ligne[:15]:
            indice=ligne.index(':')
            line=ligne[indice+1:].rstrip('\n')
            dialog_line.append(line)
        else:
            x=0

print(dialogs)

topics_dict = {"response": [], "context": [], "context0": [], "context1": [], \
               "context2": [], "context3": [], "context4": [], "context5": []}
contexted = []
n=7

for k in range(len(dialogs)):
    for i in range(n,len(dialogs[k])):
        row=[]
        prev = i - 1 - n
        for j in range(i, prev, -1):
            row.append(dialogs[k][j])
        contexted.append(row)

print(len(contexted))


columns = ['response', 'context']
columns = columns + ['context/'+str(i) for i in range(n-1)]

df = pd.DataFrame.from_records(contexted, columns=columns)
df.head(1000)

trn_df, val_df = train_test_split(df, test_size = 0.2)
trn_df.head(10000)

[["ce que je voulais te. te demander (toux) c'est .. on va parler du Marais ce quartier mais d'abord j'ai envie de te demander .. qu'est-c- où c'était habité d'abord (voix très basse qui influence le ton très bas et grave des réponses qui suivront)", "où c'était", "oui quand est-ce que tu as . que tu as habité d'abord quelque part et où quand", "mais .. bon ça m'est très difficile de répondre à ça parce que c'est une question que j'ai beaucoup trop réfléchi hein notamment à travers le texte d'Heidegger que tu connais Bâtir Habiter Penser donc je suis très mobilisée par une par une pensée théorique d'abord hein de de l'habiter je peux pas avoir une réponse naïve sur l'habiter", 'pourquoi ça', "ben parce que le le si tu veux le texte: d'Heidegger est tellement présent que je ne ferais que: de refaire un cours sur ce que j'ai fait sur Heidegger la semaine dernière bon c'est pas possible que je te réponde comme ça mais ce qui me vient c'est: comme ça comme à la mémoire c'est des flash. euh

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
1072,Les racistes sont convaincus que le groupe au...,Le raciste est quelqu’un qui se trompe.,"Oui, ma fille. Mais ce n’est pas tout. Tu as ...","Si j’ai bien compris, le raciste a peur de l’...","Et puis, si on autorisait le clonage, des hommes","Ça me fait peur… Même ma meilleure amie, je n...","Tu te rends compte, si on pouvait reproduire ...","Tu as raison, je n’aimerais pas avoir deux Cé..."
96,ah c'était complètement la plaque-tournante c'...,et le Marais c'était un peu la plaque-tournante,ou travail promis et s- et et et dès qu'ils ar...,ou pas carte mais travail,enfin ils ont tout fait pour mais: mes parents...,enfin c'est pas le hasard si ils ont tout fait,et lui je retournais le voir et je retournais ...,et lui tu retournais le voir c'était pas:
1183,"1830-1962, ça fait beaucoup de temps, cent tr...","Oui, elle est indépendante depuis 1962 ; ce s...",L’Algérie est libre…,"Pas tout un pays, mais si son gouvernement dé...",Comment un pays peut-il être raciste ?,Il est raciste et dominateur. Quand on est do...,Le colonialiste est raciste !,Au XIX
881,"Ainsi, vous n'avez pas flatté ma fantaisie dan...","Jamais aucun, monseigneur.",Et jamais aucun doute ne s'est élevé dans l'es...,"Pas la moindre, monseigneur.","Important! Oui, vous avez raison; terrible, ef...","Lui, la nourrice et moi, nous sommes les seule...","Et... il est le seul, après vous et la nourric...","Comme moi-même, monseigneur."
178,oui donc on a accepté ça et on a mangé là et b...,il était juif,je me suis mariée selon la tradition juive c'e...,tu t'es mariée selon la tradition juive,que quand je me suis mariée on a mangé dans la...,31:41),31:36),oui: mais:
...,...,...,...,...,...,...,...,...
1044,Donc il fabrique plus de mélanine que moi. Je...,"Mélanine, c’est comme un colorant.",Alors mon copaim Abdou fabrique plus de…,L’aspect foncé de la peau est dû à un pigment...,Pourquoi les Africains ont la peau noire et l...,"Si ton institutrice a vraiment dit cela, elle...","Mais, Papa, on dit bien que quelqu’un est de ...","Il n’y en a pas, mais le raciste croit ou fai..."
1095,"Je ne comprends pas, de quel amour tu parles ?","Ça, c’est la méfiance naturelle que certaines...",Et ça donne la haine ?,C’est le fait de fermer la porte et les fenêt...,"C’est quoi le refus, le rejet ?","Comme disait le général de Gaulle, « vaste","Dis-moi, Papa, comment faire pour que les gen...",Oui. Mais tous ceux qui votent pour Le Pen ne...
1130,"Oui, évidemment, nous retrouvons la peur de l...",Accuser des immigrés ne va pas lui donner du ...,Tu as raison. Mais les racistes sont des ment...,Mais les immigrés sont eux aussi frappés par le,"Il y a très longtemps, la communauté d’Israël...",C’est quoi un bouc émissaire ?,"Parfois, des gens éduqués et cultivés, à la s...",Comment ?
860,C'est ce qu'on me dit tous les ans.,"Eh bien, mon cher monsieur, cet honneur ne tar...",C'est vrai; mais je voudrais que mon grand-pèr...,"Vous aimez pourtant la parure, un peu trop même.",Un présent? Peu m'importe. Je ne suis plus un ...,Mieux que cela.,Une nouvelle! ce sera donc la première de ma v...,J'ai une conversation sérieuse à vous demander...


### Dataset Forum.fr

_Putting the data extracted from Forum.fr in the right format. Forum.fr is a forum similar to Reddit with a more friendly content. This dataset was not used due to a lack of proper data._

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


fichier = open("/content/ForumQuotidien.txt", "r",encoding="ISO 8859-1")

dialogs=[]
dialog_line=[]
content=fichier.readlines()
for ligne in content:
    ligne=ligne.replace('\n'," ")
    if "http" not in ligne:
      dialog_line.append(ligne)
    if '\n' == ligne:
      if len(dialog_line)>1:
        dialogs.append(dialog_line)
        dialog_line=[]

print(dialogs)

topics_dict = {"response": [], "context": [], "context0": [], "context1": [], \
               "context2": [], "context3": [], "context4": [], "context5": []}
contexted = []
n=7

for k in range(len(dialogs)):
    for i in range(n,len(dialogs[k])):
        row=[]
        prev = i - 1 - n
        for j in range(i, prev, -1):
            row.append(dialogs[k][j])
        contexted.append(row)

print(len(contexted))


columns = ['response', 'context']
columns = columns + ['context/'+str(i) for i in range(n-1)]

df = pd.DataFrame.from_records(contexted, columns=columns)
df.head(1000)

trn_df, val_df = train_test_split(df, test_size = 0.2)
trn_df.head(10000)

[['Merci à cette inconnue qui un matin dans une gare ma offert des fleurs, et merci aussi à leffet Impulse !1\n', 'Juste pour dire...je me nomme moi-me^me...car tant que ty es pas...tu sais pas....je rentre toujours par des chemins bucoliques et là, travaux...donc feux en pleine campagne.....bon feu rouge...je marrête....une bagnole derriére commence à faire des appels de phares....bon....rouge cest rouge!!!!!....et là le mec descend et vient taper ma vitre...jouvre sur 5 cm parce que je le trouve bizarre...mielleux...jai des frissons...bref....il mexplique quil est perdu...il cherche un bled qui est au moins à 30 bornes de moi.....mais "je suis plutôt perdu avec le GPS...ça serait bien sympa que vous veniez dans ma voiture régler mon GPS".....ben nan!!!!.....je me suis barrée vite fait.Des fois....ton amie, cest toi...et ton intuition...1\n', 'Bah... merci à celles et ceux qui ont compris le sujet du topic, hein...\n', 'Un mètre de neige, tous les transports annulés et moi, 17ans, qui

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
469,"Lorsque lon rend visite à des gens, cest dans ...","""On peut difficilement se faire un ami en un a...",La grandeur dune nation et ses progrès moraux ...,Les hommes sont si nécessairement fous que ce ...,"On a changé de jour, alors je peux... ""On ne s...",La mode est ce que lon porte soi-même. Ce qui ...,"Quand le viol est inévitable, détend toi et jo...","Quand la main désigne le but, linnocent regard..."
566,Je suis sur le sdr sur lequel il est en 1 sema...,"Sur le site de rencontre ou il est, depuis 3 j...","Bon OK, je cède à nouveau... Lhistoire de ma v...","Soyez pas triste, @Nutkin. Si personne dautre ...",Fuis moi je te suisSuis moi je te fuis\n,Contenus SponsorisésContenus SponsorisésConten...,Peut-être quil sest rendu compte quavec vou...,"Bonsoir,Peut-être quil fait ça avec toutes les..."
635,de lhiver .............jveux du soleil :smile...,Marre des dollz !(cest moi qui dit ça ! :smile...,marre des cons (ça pas été déjà dit ça? :smile...,mare de la démocratie dictatoriale\n,Marre davoir mal au crâne ! :smile2:\n,que mon chéri passe son temps sur lordinateur\n,marre de tourner en rond.......\n,mare que le lapin de ma copine narrête pas de ...
130,Merci pour ses explications^^\n,"Non pas spécialement, les chiens halètent plus...","Pour ma chienne, jai trouvé quelle tirait moin...","Ton raisonnement et logique, mais, en fait ces...","Bonjour Aupaline, tu nas pas à texcuser, au co...",Je mexcuse de minterposer dans votre discussio...,"Bonjour lko, javais déjà pris ce collier mais ...","Bonjour, bonne idée ce topic, sinon jai un ber..."
691,"Fleurs de cerisier, abeilles qui butinent. Trè...",Musique très agréable: \n,Lac et montagne: \n,Symphonie deau vive et de chants doiseaux: \n,Un chêne à la croisée des chemins dans une for...,Paysages de rêve: \n,Pour les enfants... ou les grands \n,"Original avec un petit vélo, et en plus une mu..."
...,...,...,...,...,...,...,...,...
821,moi1\n,MOI !Je suis géniale !!!Je suis extraordinaire...,"je dirais mon enfant, plus que mon compagnon, ...","Bonjour,Cest avant tout ma mère ! Cest la pr...",je suis narcissique .\n,Parrait quil faut saimer soi-même avant daimer...,Ce sont de bonnes personnes qui vont nous appo...,et après tu revend linfo a Amazon ou a Tik Tok...
774,Que la Tremblade se situait en dessous de lest...,Même ici certains ont plus se certitudes que l...,"Un jour, un moniteur de colo a fait passer tou...",Quand ZEMMOUR parle déducation et de pédagogie...,"Jai regardé dernièrement le film ""mensonge et ...",=)Anecdote littéraire remontant à quelques ann...,"Au salon de coiffure , la cliente a coté de mo...",Un démarcheur vient me proposer ses offres pou...
451,"Il nexiste que deux choses infinies, lunivers ...",Les mots sont comme les sacs : ils prennent la...,"si lon batissait la maison du bonheur , la plu...","Mourir, ce nest rien. Commence donc par vivre....","Des,victimes,ont,en,est,tous....Edgard.A.POE.....",Je laisse à votre réflexion cette citation de ...,"Rien nest vrai, rien nest faux,tout est songe ...","Un roman est comme un archet, la caisse du vio..."
83,"Hello,Merci pour ce récap, cest toujours utile...",Jété un peu déçu de découvrir que mon test de ...,Suite à un oublie de pilule en période dovulat...,"Jai peur dune seconde grossesse, méme si je ve...","Comment peux-tu dire que cest berk, si tu nas ...","Ah non pas de copine, merci, je ne suis pas ho...",pour linstant aucun risque que tu sois enceint...,"Bonjour! Comme je te comprends, jai aussi très..."


### Model

_Implementation of the model based on this [script](https://colab.research.google.com/drive/15wa925dj7jvdvrz8_z3vU7btqAFQLVlG#scrollTo=7KrNfVNueNhR ) using HuggingFace. We use basic configuration and the datasets we formatted._

In [ ]:
! pip -q install transformers

from transformers import AutoModelWithLMHead, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelWithLMHead.from_pretrained("microsoft/DialoGPT-small")
"""
Fine-tuning the library models for language modeling on a text file (GPT, GPT-2, BERT, RoBERTa).
GPT and GPT-2 are fine-tuned using a causal language modeling (CLM) loss while BERT and RoBERTa are fine-tuned
using a masked language modeling (MLM) loss.
"""

import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange

from pathlib import Path

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)


try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

# Configs
logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

     |████████████████████████████████| 2.5 MB 31.5 MB/s 
     |████████████████████████████████| 3.3 MB 65.3 MB/s 
     |████████████████████████████████| 895 kB 51.0 MB/s 


/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:847: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
# Args to allow for easy convertion of python script to notebook
class Args():
    def __init__(self):
        self.output_dir = 'output-small'
        self.model_type = 'gpt2'
        self.model_name_or_path = 'microsoft/DialoGPT-small'
        self.config_name = 'microsoft/DialoGPT-small'
        self.tokenizer_name = 'microsoft/DialoGPT-small'
        self.cache_dir = 'cached'
        self.block_size = 512
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 4
        self.per_gpu_eval_batch_size = 4
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 3
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 3500
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = False
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False
        self.fp16_opt_level = 'O1'

args = Args()

In [ ]:
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

class ConversationDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):

        block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

        directory = args.cache_dir
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size)
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)

            self.examples = []
            for _, row in df.iterrows():
                conv = construct_conv(row, tokenizer)
                self.examples.append(conv)

            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)

In [ ]:
# Cacheing and storing of data/checkpoints

def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
    return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
        shutil.rmtree(checkpoint)

### Training

In [ ]:
def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))
    # add_special_tokens_(model, tokenizer)


    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

# Evaluation of some model

def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [ ]:
# Main runner

def main(df_trn, df_val):
    args = Args()
    
    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup CUDA, GPU & distributed training
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    model = AutoModelWithLMHead.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir=args.cache_dir,
    )
    model.to(args.device)
    
    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)

        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelWithLMHead.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelWithLMHead.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, df_trn, df_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results

In [ ]:
main(trn_df, val_df)

07/20/2021 08:22:52 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
07/20/2021 08:22:52 - INFO - filelock -   Lock 140410976495632 acquired on cached/0cbdd50f204f3ddbaa452e976340a5725f0b5ddb201704058c87e14d9679e070.e6898db50ba3aa698f0f652e876a1e4bd813321dea3e22b776f9a3c39d36aaab.lock


07/20/2021 08:22:52 - INFO - filelock -   Lock 140410976495632 released on cached/0cbdd50f204f3ddbaa452e976340a5725f0b5ddb201704058c87e14d9679e070.e6898db50ba3aa698f0f652e876a1e4bd813321dea3e22b776f9a3c39d36aaab.lock
07/20/2021 08:22:52 - INFO - filelock -   Lock 140410957358992 acquired on cached/5f8cf488e0bdda2e393e798f478673a4d26c1386082a1a05e42269f3ecc89f50.67d01b18f2079bd75eac0b2f2e7235768c7f26bd728e7a855a1c5acae01a91a8.lock


07/20/2021 08:22:52 - INFO - filelock -   Lock 140410957358992 released on cached/5f8cf488e0bdda2e393e798f478673a4d26c1386082a1a05e42269f3ecc89f50.67d01b18f2079bd75eac0b2f2e7235768c7f26bd728e7a855a1c5acae01a91a8.lock
07/20/2021 08:22:52 - INFO - filelock -   Lock 140410957358608 acquired on cached/3cf340c89a43b5e6f31c4cd609fc2fc92f3d7aafdf6c8987e2ea9e02cb78b4e2.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f.lock


07/20/2021 08:22:52 - INFO - filelock -   Lock 140410957358608 released on cached/3cf340c89a43b5e6f31c4cd609fc2fc92f3d7aafdf6c8987e2ea9e02cb78b4e2.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f.lock
07/20/2021 08:22:52 - INFO - filelock -   Lock 140410966505104 acquired on cached/4e3f74e7c741909c4d1b48a23febe75c1be66a20c2b98cf7db4b8b10f12dc10c.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


07/20/2021 08:22:53 - INFO - filelock -   Lock 140410966505104 released on cached/4e3f74e7c741909c4d1b48a23febe75c1be66a20c2b98cf7db4b8b10f12dc10c.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


...

07/20/2021 08:23:14 - INFO - __main__ -   Training/evaluation parameters <__main__.Args object at 0x7fb35d5030d0>
07/20/2021 08:23:14 - INFO - __main__ -   Creating features from dataset file at cached
Token indices sequence length is longer than the specified maximum sequence length for this model (2278 > 1024). Running this sequence through the model will result in indexing errors
07/20/2021 08:23:36 - INFO - __main__ -   Saving features into cached file cached/gpt2_cached_lm_512
07/20/2021 08:23:36 - INFO - __main__ -   ***** Running training *****
07/20/2021 08:23:36 - INFO - __main__ -     Num examples = 5487
07/20/2021 08:23:36 - INFO - __main__ -     Num Epochs = 3
07/20/2021 08:23:36 - INFO - __main__ -     Instantaneous batch size per GPU = 4
07/20/2021 08:23:36 - INFO - __main__ -     Total train batch size (w. parallel, distributed & accumulation) = 4
07/20/2021 08:23:36 - INFO - __main__ -     Gradient Accumulation steps = 1
07/20/2021 08:23:36 - INFO - __main__ -     Total

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:247: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


07/20/2021 08:42:11 - INFO - __main__ -   Saving model checkpoint to output-small/checkpoint-3500
07/20/2021 08:42:16 - INFO - __main__ -   Saving optimizer and scheduler states to output-small/checkpoint-3500
07/20/2021 08:45:15 - INFO - __main__ -    global_step = 4068, average loss = 2.370806556418465
07/20/2021 08:45:15 - INFO - __main__ -   Saving model checkpoint to output-small


07/20/2021 08:45:18 - INFO - __main__ -   Evaluate the following checkpoints: ['output-small']
07/20/2021 08:45:20 - INFO - __main__ -   Creating features from dataset file at cached
Token indices sequence length is longer than the specified maximum sequence length for this model (4707 > 1024). Running this sequence through the model will result in indexing errors
07/20/2021 08:45:22 - INFO - __main__ -   Saving features into cached file cached/gpt2_cached_lm_512
07/20/2021 08:45:22 - INFO - __main__ -   ***** Running evaluation  *****
07/20/2021 08:45:22 - INFO - __main__ -     Num examples = 1372
07/20/2021 08:45:22 - INFO - __main__ -     Batch size = 4


RuntimeError: ignored

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small') # one can choose DialoGPT-medium depeding on the version you trained
model = AutoModelWithLMHead.from_pretrained('output-small') # one can choose the path of a pre-trained output located in the [...] directory

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # print(new_user_input_ids)

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids, max_length=200,
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=3,       
        do_sample=True, 
        top_k=100, 
        top_p=0.7,
        temperature = 0.8
    )
    
    # pretty print last ouput tokens from bot
    print("FrenchBot: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:847: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


>> User:Bonjour
FrenchBot: Merci! 
>> User:Quoi N
FrenchBot: Toujours le temps? 
>> User:Le temps de quoi ?
FrenchBot: Bonjours, on peut avoir le cas mais on a une fois que le tempière dans les vieux qui sont pas de faire des hommages?  
>> User:C'est n"importe quoi
FrenchBot: J'ai une bonne fois mais il faut comme ça. 
>> User:Ca veut rien dire
FrenchBot: !!!!!!!!!! 
